In [4]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import os
import pandas as pd
from PIL import Image  #Python的图像处理库
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt
import gc



In [5]:
IMG_SIZE =224
def read_pic_data_from_file(DIR):
    pic_df = []
    for i in os.listdir(DIR):
        path = os.path.join(DIR,i) #找到图片路径
        img = Image.open(path)  #根据路径打开图片
        #img = img.convert('L')   
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        pic_info = np.array(img)
        pic_df.append(pic_info)
    return pic_df



In [19]:
DIR1 = 'C:/Users/zhuhuiling/Desktop/0121/扩增/no_damage'
good_car_df = read_pic_data_from_file(DIR1) 


In [20]:
DIR1 = 'C:/Users/zhuhuiling/Desktop/0121/扩增/other'
DIR2 = "C:/Users/zhuhuiling/Desktop/0121/0118_汇总/other_damage_together_286"
other_damage_df = read_pic_data_from_file(DIR1) + read_pic_data_from_file(DIR2) 

In [21]:
DIR1 =  'C:/Users/zhuhuiling/Desktop/0121/扩增/scra'
DIR2 =  "C:/Users/zhuhuiling/Desktop/0121/0118_汇总/scratch_together_509"
scrath_df = read_pic_data_from_file(DIR1) +  read_pic_data_from_file(DIR2) 

In [22]:
# 合并三类作为train_x
df_total = good_car_df + other_damage_df + scrath_df

# 创建标签作为train_y
label = ['good_car'] * len(good_car_df) + ['other_damage'] * len(other_damage_df) + ['scratch'] * len(scrath_df)

# 标签必须转化为数值
def label_img(name):
    if name == 'good_car': return np.array([1, 0, 0])
    elif name == 'other_damage' : return np.array([0, 1,0 ])
    elif name == 'scratch': return np.array([0, 0, 1])

pic_label = []
for i in label:
    pic_label.append(label_img(i))
       
# 将x 与 y 对应
train_data = list(zip(df_total,pic_label))

import random
random.shuffle(train_data)


In [23]:
train_data_new = []
for i in train_data:
    if i[0].shape == (224, 224, 3):
        train_data_new.append(i)

In [24]:
len(train_data_new)

6783

In [9]:
# reshape数据，并且确保数据为np.array格式
trainImages = np.array([i[0] for i in train_data_new]).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
trainLabels = np.array([i[1] for i in train_data_new])

trainImages = np.array(trainImages)
trainLabels = np.array(trainLabels)

# 划分验证集
train_x = trainImages[0:600,] 
train_y = trainLabels[0:600,] 

val_x = trainImages[600:800:,] 
val_y = trainLabels[600:800:,] 

test_x = trainImages[800:,] 
test_y = trainLabels[800:,] 

In [10]:
train_x = train_x/225
test_x = test_x/225
val_x = val_x/225

In [66]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from pdb import set_trace as dd
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input


# inceptionV3

In [69]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)


In [70]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

In [71]:
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized), i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers: layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [73]:
#model.fit(train_x, train_y, batch_size = 50, epochs = 300, verbose = 1,validation_data=(val_x,val_y)) 

# ResNet50

In [76]:
# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False)


94658560/94653016 [==============================] - 1510s 16us/step


In [77]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

In [78]:
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized), i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers: layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [79]:
model.fit(train_x, train_y, batch_size = 50, epochs = 300, verbose = 1,validation_data=(val_x,val_y)) 

Train on 600 samples, validate on 200 samples
Epoch 1/300
Train on 600 samples, validate on 200 samples
Epoch 1/300
200/600 [=========>....................] - ETA: 4:47 - loss: 6.1566 - acc: 0.4650

KeyboardInterrupt: 

KeyboardInterrupt: 

# VGG16

In [81]:
from keras.applications.vgg16 import VGG16

In [ ]:
?VGG16()

In [82]:
model_base = VGG16(weights='imagenet', include_top=False)

In [83]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

In [87]:
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized), i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers: layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [88]:
#model.fit(train_x, train_y, batch_size = 50, epochs = 300, verbose = 1,validation_data=(val_x,val_y)) 